**Reload modules when executing code**

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Importando dados**

In [14]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from salicml.utils.dates import Dates
from salicml.models.projects import Projects

PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

**Carregar a tabela de Projetos**

In [15]:
cols = ['IdPRONAC', 'DtInicioExecucao', 'DtFimExecucao']
projects_csv = os.path.join(DATA_FOLDER, 'projetos.csv')

projects = Projects(projects_csv)

df_projects = projects.get_data_frame_columns(cols)
display(df_projects.head())

,IdPRONAC,DtInicioExecucao,DtFimExecucao
0,1,2010-01-01 00:00:00,2010-12-31 00:00:00
1,2,NaN,NaN
2,3,NaN,NaN
3,4,NaN,NaN
4,5,NaN,NaN


**Projetos na etapa de Admissibilidade**

In [16]:
filter_with_start_execution = pd.notna(df_projects['DtInicioExecucao'])
filter_currentdate_less_than_start_execution = time.strftime("%Y-%m-%d %H:%M:%S") < df_projects['DtInicioExecucao']

filter_admissibility = filter_with_start_execution & filter_currentdate_less_than_start_execution

df_projects_admissibility = df_projects[filter_admissibility]

display("Projetos na etapa de Admissibilidade: {}".format(len(df_projects_admissibility)))
display(df_projects_admissibility.head())

'Projetos na etapa de Admissibilidade: 476'

,IdPRONAC,DtInicioExecucao,DtFimExecucao
53776,53777,4201-04-25 00:00:00,NaN
63137,63138,4201-04-07 00:00:00,NaN
158959,160167,2113-03-01 00:00:00,2114-03-01 00:00:00
197648,203930,2019-04-01 00:00:00,2019-12-31 00:00:00
197902,205207,2018-06-18 00:00:00,2019-11-18 00:00:00


**Projetos na etapa de Prestação de Contas**

In [17]:
filter_with_finish_execution = pd.notna(df_projects['DtFimExecucao'])
filter_currentdate_bigger_than_start_execution = time.strftime("%Y-%m-%d %H:%M:%S") > df_projects['DtFimExecucao']

filter_accountability = filter_with_finish_execution & filter_currentdate_bigger_than_start_execution

df_projects_accountability = df_projects[filter_accountability]

display("Projetos na etapa de Prestação de Contas: {}".format(len(df_projects_accountability)))
display(df_projects_accountability.head())

'Projetos na etapa de Prestação de Contas: 68808'

,IdPRONAC,DtInicioExecucao,DtFimExecucao
0,1,2010-01-01 00:00:00,2010-12-31 00:00:00
9,10,NaN,2012-01-30 00:00:00
22,23,2011-05-02 00:00:00,2015-12-31 00:00:00
28,29,2017-12-08 00:00:00,2018-01-10 00:00:00
35,36,2003-01-01 00:00:00,2003-09-30 00:00:00
